In [1]:
import pandas as pd
import os
import sys
import datetime
import urllib
import numpy as np
import cv2
import pymysql
import json

from pathlib import Path
from enum import Enum
from io import BytesIO, StringIO
from typing import Union

from detection import upload_to_gcs
from detection import generate_download_signed_url_v4
from detection import get_similar_products_uri
from detection import query_product

In [2]:
project_id = 'abstract-veld-289612'
location = 'asia-east1'
product_set_id = 'PAIR-Filter'
bucket_name = 'ftmle'
product_category= 'homegoods-v2'

In [3]:
def search_product(bucket_name, blob_name):
    serving_url = generate_download_signed_url_v4(bucket_name, blob_name)
    res = get_similar_products_uri(project_id, location, product_set_id, product_category,
        image_uri=serving_url, filter='')
    return res, serving_url

In [19]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='35.221.181.94',
    port=int(3306),
    user="mkhoa",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()

def query_room():
    '''
    
    '''
    query = '''
    SELECT a.id as 'room_id', a.subtitle as 'room_type', b.bucket_path
    FROM project.DesignsDetail a
    LEFT JOIN Files b ON a.id = b.id
    LEFT JOIN design2item c ON a.id = c.room_id
    WHERE b.bucket_path like '%/Design/%' 
    AND (a.subtitle like '%Living%' OR a.subtitle like '%Dining%' OR a.subtitle like '%Bedroom%')
    AND c.room_id is null
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY SELECT:', err)
    result = cur.fetchall()
    return result

def insert_design2item(room_id, idx_results, product_id, score):
    '''
    
    '''
    query = f'''
    INSERT INTO design2item(room_id, idx_results, product_id, score)
    VALUES('{room_id}', '{idx_results}', '{product_id}', '{score}');
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY INSERT:', err)

In [20]:
room = pd.DataFrame(query_room(), columns=['room_id', 'room_type', 'bucket_path'])

In [21]:
room

,room_id,room_type,bucket_path
0,6017,"Transitional Living Room, Orange County",Images/Design/6017-home-design.jpg
1,9730,"Contemporary Living Room, Minneapolis",Images/Design/9730-home-design.jpg
2,9811,"Transitional Living Room, Toronto",Images/Design/9811-home-design.jpg
3,20273,"Modern Dining Room, Dallas",Images/Design/20273-home-design.jpg
4,20516,"Craftsman Dining Room, Boston",Images/Design/20516-new-hampshire-lake-house-c...
...,...,...,...
5517,29337,"Modern Bedroom, Hawaii",Images/Design/29337-kaikoo-place-hawaii-modern...
5518,6287,"Traditional Living Room, Charlotte",Images/Design/6287-home-design.jpg
5519,20540,"Modern Dining Room, New York",Images/Design/20540-tribeca-penthouse-lucy-har...
5520,22721,Traditional Dining Room,Images/Design/22721-tudor-revival-estate-full-...


In [ ]:
for idx, row in room.iterrows():
    blob = row['bucket_path']
    room_id = row['room_id']
    res, serving_uri = search_product(bucket_name, blob)
    if idx % 100 == 0:
        print(idx)
    for i in range(len(res)):
        for j in range(len(res[i].results)):
            try:
                product_id = res[i].results[j].product.name.split('/')[-1]
                score = res[i].results[j].score
                insert_design2item(room_id, i, product_id, score)
            except:
                print(room_id)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [136]:
result = []
for i in range(len(res)):    
    for j in range(len(res[i].results)):
        d = {}
        d['object'] = i
        d['idx'] = int(res[i].results[j].product.name.split('/', -1)[-1])
        d['name'] = res[i].results[j].product.display_name
        d['score'] = res[i].results[j].score
        result.append(d)

In [137]:
df = pd.DataFrame(result)

In [131]:
def query_product(id):
    '''
    
    '''
    query = f'''
    SELECT id, website, product, image, url, b.group_category 
    FROM project.ProductHeader a
    LEFT JOIN project.category b ON a.category = b.Category
    WHERE id = '{id}'
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY INSERT:', err)
    result = cur.fetchone()
    return result

In [135]:
query = pd.DataFrame(df['idx'].apply(query_product).tolist(), columns=['idx', 'Website', 'Product Name', 'Image', 'URL', 'Category'])

In [142]:
result = df.merge(query)

In [148]:
for i in result['Category'].unique():
    print(i)

Bench
Object
Pillows
Table
Chair
Bed
Sofa
Lighting
